In [1]:
import pandas as pd
raw_data = pd.read_csv('train_2022.csv')
raw_data

,row_id,TEXT,LABEL
0,0,director dirk shafer and co-writer greg hinton...,0
1,1,"a charming , quirky and leisurely paced scotti...",1
2,2,"the price was good , and came quickly though ...",1
3,3,i was looking forward to this game for a coupl...,0
4,4,arguably the year 's silliest and most incoher...,0
...,...,...,...
1995,1995,an imaginative comedy\/thriller .,1
1996,1996,a savvy exploration of paranoia and insecurity...,1
1997,1997,on the other hand for power grating you ve got...,1
1998,1998,"like dickens with his passages , mcgrath craft...",1


In [2]:
from transformers import DebertaForSequenceClassification, DebertaTokenizer

model_name = "microsoft/deberta-base"
tokenizer = DebertaTokenizer.from_pretrained(model_name)
model = DebertaForSequenceClassification.from_pretrained(model_name)

D:\Anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


D:\Anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
%%time
from transformers import DebertaTokenizer, DebertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.nn.utils.rnn import pad_sequence

# 載入預訓練的 DeBERTa tokenizer 和模型
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=2)  # 2 表示二分類，正向和負向情感

# 轉換成 DataFrame
train_data = raw_data.copy()

# 使用 tokenizer 將文本轉換為 token IDs
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt')['input_ids'][0]

train_data['input_ids'] = train_data['TEXT'].apply(tokenize_text)

# 轉換成可以輸入模型的格式
inputs = pad_sequence(train_data['input_ids'].tolist(), batch_first=True)

# 將資料拆分為訓練集和測試集
train_inputs, test_inputs, train_labels, test_labels = train_test_split(inputs, train_data['LABEL'].tolist(), test_size=0.2)

# 創建 PyTorch DataLoader
train_dataset = TensorDataset(train_inputs, torch.tensor(train_labels))
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# 定義 optimizer 和損失函數
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# 訓練模型
model.train()
for epoch in range(5):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

# 評估模型
model.eval()
with torch.no_grad():
    outputs = model(test_inputs)
    predicted_labels = torch.argmax(outputs.logits, dim=1).tolist()

# 計算分類報告
report = classification_report(test_labels, predicted_labels)
print(report)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
D:\Anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


              precision    recall  f1-score   support

           0       0.80      0.76      0.78       204
           1       0.76      0.81      0.78       196

    accuracy                           0.78       400
   macro avg       0.78      0.78      0.78       400
weighted avg       0.78      0.78      0.78       400



In [10]:
test_data = pd.read_csv('test_no_answer_2022.csv')

# Tokenize and pad test data
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt')['input_ids'][0]

test_data['input_ids'] = test_data['TEXT'].apply(tokenize_text)
test_inputs = pad_sequence(test_data['input_ids'].tolist(), batch_first=True)

# Create DataLoader for test data
test_dataset = TensorDataset(test_inputs)
test_loader = DataLoader(test_dataset, batch_size=8)

# Evaluate the model on test data
model.eval()
predicted_labels = []

with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs[0])  # inputs[0] contains the input_ids
        predicted_labels.extend(torch.argmax(outputs.logits, dim=1).tolist())
        
test_data['PREDICTED_LABEL'] = predicted_labels

# 保存預測結果到 CSV 文件
test_data.to_csv('predicted_results.csv', index=False)

# 打印預測結果
print(test_data[['TEXT', 'PREDICTED_LABEL']])

                                                    TEXT  PREDICTED_LABEL
0       good to know if you can t find these elsewhere .                1
1      love it !  the grill plates come out and pop i...                1
2      i m convinced this was a poorly executed refur...                0
3      i would never have complained about that if it...                0
4      the photo shows the same whole ,  large candie...                1
...                                                  ...              ...
10995             i didn t quite get it the first time .                0
10996  i ve tried installing with and without the oem...                1
10997  i was parked at a truck stop in the cincinnati...                1
10998  i recently bought this case after seeing some ...                1
10999  the keyboard types only % of the time and the ...                0

[11000 rows x 2 columns]


In [6]:
predictions

tensor([0])